In [1]:
%pylab inline
import numpy as np
import time
import threading
import pywren
from importlib import reload
import threading

Populating the interactive namespace from numpy and matplotlib


In [2]:

reload(pywren)
wrenexec = pywren.local_executor()

In [3]:
def foo(x):
    return x + 1
f = wrenexec.call_async(foo, 1.0)


{'storage_config': {'storage_backend': 's3', 'storage_prefix': 'pywren.jobs', 'backend_config': {'bucket': 'jonas-pywren-604', 'region': 'us-west-2'}}, 'func_key': 'pywren.jobs/5e98a75f-0201-476c-885a-289335d8b7ec/func.pickle', 'data_key': 'pywren.jobs/5e98a75f-0201-476c-885a-289335d8b7ec/aggdata.pickle', 'output_key': 'pywren.jobs/5e98a75f-0201-476c-885a-289335d8b7ec/00000/output.pickle', 'status_key': 'pywren.jobs/5e98a75f-0201-476c-885a-289335d8b7ec/00000/status.json', 'callset_id': '5e98a75f-0201-476c-885a-289335d8b7ec', 'job_max_runtime': 300, 'data_byte_range': (0, 11), 'call_id': '00000', 'use_cached_runtime': True, 'runtime': {'s3_bucket': 'pywren-public-us-west-2', 's3_key': 'pywren.runtimes/default_3.6.meta.json', 'runtime_storage': 's3'}, 'pywren_version': '0.3.0', 'runtime_url': 's3://pywren-public-us-west-2/e1e82c-pywren.runtimes/default_3.6.tar.gz.0038', 'host_submit_time': 1505258161.025185}
local_task_run_dir= /tmp/task/0
CWD IS /private/tmp/task/0 we just chdir'd to /t

In [4]:
f.result()

StorageOutputNotFoundError: Output for 3fd2f8a5-5213-4f27-ad3d-a612b44ea9c9 00000 not found in storage.

In [76]:
ITER_GRANULARITY_SEC = 0.1


class IterFuture(object):
    def __init__(self, method, max_iters, *args):
        self.max_iters = max_iters
        self.args = args
        self.thread = threading.Thread(target=self.run_iterations, 
                                       args=[method, max_iters] + list(args))
        self.thread.start()
        
        self._iteration = -1
        
    def run_iterations(self, method, max_iters, *args):
        old_state = None
        for i in range(max_iters+1):
            old_state = method(i, old_state, *args)
            time.sleep(ITER_GRANULARITY_SEC)
            self._iteration = i
        self._result = old_state
        return old_state
    
    @property
    def done(self):
        is_done = self._iteration == (self.max_iters)
        if is_done:
            self.thread.join()
        return is_done
    
    @property
    def iter(self):
        return self._iteration
    
        
def itermap(method, max_iters, args):
    return [IterFuture(method, max_iters, arg) for arg in args]



In [77]:
N = 10

def my_iterative_method(iter_k, current_state):
    if iter_k == 0:
        x_next = np.random.rand(N)
    else:
        x_next = current_state + 1.0 #  alpha * grad(x_next)
    return x_next


iter_future = IterFuture(my_iterative_method, 100)
while not iter_future.done:
    print(iter_future.iter)
    time.sleep(1)
    
    

-1
8
18
28
37
47
57
67
76
86
96


In [78]:
N = 100

def my_iterative_method(iter_k, current_state, arg):
    if iter_k == 0:
        x_next = np.random.rand(N) + arg
    else:
        x_next = current_state + 1.0 #  alpha * grad(x_next)
    return x_next

iterfutures = itermap(my_iterative_method, 100, 10 * np.arange(4))



In [87]:
IN_NOTEBOOK = True

def iterfutures_wait(futures,notebook=False):
    
    if notebook:
        from tqdm import tqdm_notebook as tqdm
    else:
        from tqdm import tqdm

    total_max_iter = np.sum([f.max_iters for f in futures])
    last_done_iters = 0
    with tqdm(total=total_max_iter) as pbar:
        while True:
            total_done_iters = np.sum([max(f.iter, 0) for f in futures])
            delta_iters = total_done_iters - last_done_iters
            last_done_iters = total_done_iters
            pbar.update(delta_iters)
            
            if np.array([f.done for f in futures]).all():
                break
            time.sleep(1)

In [89]:
iterfutures = itermap(my_iterative_method, 100, 10 * np.arange(4))
iterfutures_wait(iterfutures, IN_NOTEBOOK)

In [ ]:
# how do to do this with pywren as a backing? The interesting thing is that we are repeatedly invoking
# the same function, and just using different data

N = 10

def my_gradient_method(k, x_k, alpha):
    if k == 0:
        # init case, create initial state, do setup
        return np.zeros(N)
    else:
        x_next =  x_k + alpha * grad_func(x_k)
        return x_next
       
with iter_runner as IR:
    total_iters = 100
    alphas = [0.001, 0.01, 0.1]
    futures = IR.itermap(my_gradient_method, total_iters, alphas)
    
    IR.wait(futures) # this takes a reallllllly long time. Maybe prints a prog bar? 
    
    results = [f.result() for f in futures]


why give the checkpoints names? instead autogenerate tnems! 

you're going to have to pick locations for checkpoint results regardless, right? 



In [ ]:
                

N = 10

def my_gradient_method(k, x_k, alpha):
    if k == 0:
        # init case, create initial state, do setup
        return np.zeros(N)
    else:
        x_next =  x_k + alpha * grad_func(x_k)
        return x_next
       
with iter_runner as IR: # spins off an in-process thread
    total_iters = 100
    alphas = [0.001, 0.01, 0.1]
    futures = IR.itermap(my_gradient_method, total_iters, alphas, 
                        checkpoint_names)
    
    IR.wait(futures) # this takes a reallllllly long time. Maybe prints a prog bar? 
    
    results = [f.result() for f in futures]
